# Unmapped MARC fields and subfields
This notebook:
* Documents MARC fields and subfields omitted from Argot mappings
* Identifies MARC mapping decisions to be made, by analyzing current MARC specification (from http://www.loc.gov/marc/bibliographic/ecbdlist.html), Argot mappings documentation, and our mapping omission decisions

First, run script to generate JSON hash of current MARC specification. If successful: 
* Out will = true
* `marc_definition.json` is written to `data-documentation/marc` folder

In [7]:
require './extract_marc_bib_spec'

true

In [2]:
require 'json'
require 'simple_xlsx_reader'
doc = SimpleXlsxReader.open('MARC_subfields_not_mapped.xlsx')
data = doc.sheets.first.data
puts doc.sheets.first.headers


["MARC tag", "Subfield", "Field name", "Omission type", "Reason category", "More details"]


In [12]:
data_hash = data.map do |r|
  { r[0].rjust(3,'0') => { 
      r[1] => {
    "omission_type" => r[3],
    "omission_reason_category" => r[4],
    "omission_details" => r[5]
      }
    }
  }
end

open('unmapped_marc_subfields.json', 'w'){ |f| f.puts JSON.pretty_generate(data_hash)}
